In [ ]:
from ase.build import molecule
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import Vibrations
from firecode.optimization_methods import Opt_func_dispatcher
from firecode.utils import HiddenPrints
from ase.optimize import LBFGS

atoms = molecule('CH3CH2O')
mult = 1
dispatcher = Opt_func_dispatcher('TBLITE')
atoms.calc = dispatcher.get_ase_calc('GFN2-xTB', solvent=None)

opt = LBFGS(atoms)
with HiddenPrints():
    opt.run(fmax=1E-6)

potentialenergy = atoms.get_potential_energy()
print(potentialenergy)

In [ ]:
vib = Vibrations(atoms,
                #  indices=(0, 1),
                #  indices=tuple(i for i in range(len(atoms.get_atomic_numbers()))),
                 )
# with HiddenPrints():
vib.run()
vib_energies = vib.get_energies()
vib.get_frequencies()

In [ ]:
vib.summary()

In [ ]:
thermo = IdealGasThermo(
    vib_energies=vib_energies,
    potentialenergy=potentialenergy,
    atoms=atoms,
    geometry='nonlinear',
    symmetrynumber=1,
    spin=(mult-1)/2
)
G = thermo.get_gibbs_energy(temperature=298.15, pressure=101325.0)

In [ ]:
thermo.get_entropy(temperature=298.15, pressure=101325.0)

In [ ]:
from ase import Atoms
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import Vibrations

from firecode.optimization_methods import Opt_func_dispatcher
from firecode.units import EH_TO_EV, EH_TO_KCAL, EV_TO_KCAL
from firecode.utils import HiddenPrints, read_xyz
import shutil
import os
from ase.optimize import LBFGS


def ase_get_free_energy(
        coords,
        atomnos,
        ase_calc,
        energy,
        charge,
        mult,
        temp_C=25,
        title="temp",
        write_log=True,

    ):
    """
    energy: kcal/mol
    """

    atoms = Atoms(atomnos, positions=coords)

    ase_calc.charge = charge
    ase_calc.multiplicity = mult
    ase_calc.verbosity = 0        
    atoms.calc = ase_calc

    vib = Vibrations(atoms)

    with open(f'vib_{title}.out', 'w') as f:

        with HiddenPrints():

            f.write(f'--> FIRECODE ASE Frequency calculation report\n\n')

            # tighten convergence
            f.write(f'--> Tightening convergence to fmax=1E-4\n')
            opt = LBFGS(atoms)
            opt.run(fmax=1E-4)

            # remove cache folder
            if 'vib' in os.listdir():
                shutil.rmtree(os.path.join(os.getcwd(), 'vib'))

            # run vibrational analysis
            vib.run()
            vib_energies = vib.get_energies()
            vib.summary(log=f)

            
            thermo = IdealGasThermo(            
                vib_energies=vib_energies,
                potentialenergy=energy / EV_TO_KCAL,
                atoms=atoms,
                spin=(mult-1)/2,

                # assuming all inputs are C1 for now...
                geometry='nonlinear',
                symmetrynumber=1,
                ignore_imag_modes=True,  
            )

            EE = energy / EV_TO_KCAL / EH_TO_EV
            G = thermo.get_gibbs_energy(temperature=(temp_C + 273.15), pressure=101325.0) / EH_TO_EV
            H = thermo.get_enthalpy(temperature=(temp_C + 273.15)) / EH_TO_EV
            S = thermo.get_entropy(temperature=(temp_C + 273.15), pressure=101325.0) / EH_TO_EV
            gcorr = G - EE


            f.write('\n--> What follows is an ORCA output mock-up for scraping:\n\n')
            f.write(f'FINAL SINGLE POINT ENERGY {EE:.8f} Eh\n')
            f.write(f'G-E(el) ... {gcorr:.8f} Eh     {gcorr*EH_TO_KCAL:.2f} kcal/mol\n')
            f.write(f'Total enthalpy ... {H:.8f} Eh\n')
            f.write(f'Final entropy term ... {S:.8f} Eh\n')

    del vib
    shutil.rmtree(os.path.join(os.getcwd(), 'vib'))

    if not write_log:
        shutil.rmtree(os.path.join(os.getcwd(), f'vib_{title}.out'))

    return G * EV_TO_KCAL

mol = read_xyz('/mnt/c/Users/Nick/Desktop/Computations/toluene.xyz')
dispatcher = Opt_func_dispatcher('TBLITE')
ase_calc = dispatcher.get_ase_calc('GFN2-xTB')
ase_get_free_energy(mol.atomcoords[0], mol.atomnos, ase_calc=ase_calc, energy=0, charge=0, mult=1)

2.205021834186285